## 字符串和文本
>本章节使用的模块：
1. re
 - re.split()
 - re.match()
 - re.compile()
2. fnmatch
 - fnmatch.fnmatch()
 - fnmatch.fnmatchcase()
3. unicodedata
 - unicodedata.normalize()
4. textwrap
 - textwrap.fill()
5. sys
6. os

#### 1. 使用多个界定符分割字符串
string 对象的 split() 方法只适应于非常简单的字符串分割情形， 它并不允许有多个分隔符或者是分隔符周围不确定的空格。 当你需要更加灵活的切割字符串的时候，最好使用 re.split() 方法：

In [26]:
line = 'asdf fjdk; afed, fjek,asdf, foo'
import re
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

获取分隔符，用于其他操作：

In [2]:
line = 'asdf fjdk; afed, fjek,asdf, foo'
import re
fields = re.split(r'(;|,|\s)\s*', line)
values = fields[::2]
delimiters = fields[1::2] + ['']
print(values)
print(delimiters)
# Reform the line using the same delimiters
print(''.join(v+d for v,d in zip(values, delimiters)))

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']
[' ', ';', ',', ',', ',', '']
asdf fjdk;afed,fjek,asdf,foo


#### 2.用Shell通配符匹配字符串
fnmatch() 函数使用底层操作系统的大小写敏感规则(不同的系统是不一样的)来匹配模式。  
如果你的代码需要做文件名的匹配，最好使用 glob 模块。

In [3]:
from fnmatch import fnmatch, fnmatchcase
fnmatch('foo.txt', '*.txt')
fnmatch('foo.txt', '?oo.txt')
fnmatch('Dat45.csv', 'Dat[0-9]*')
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

#### 3.最短匹配模式
你正在试着用正则表达式匹配某个文本模式，但是它找到的是模式的最长可能匹配。 而你想修改它变成查找最短的可能匹配。  
**`*是贪婪的，会匹配最长的`**,通过在 * 或者 + 这样的操作符后面添加一个 ? 可以强制匹配算法改成寻找最短的可能匹配。

In [4]:
str_pat = re.compile(r'"(.*)"')
text1 = 'Computer says "no."'
print(str_pat.findall(text1))

text2 = 'Computer says "no." Phone says "yes."'
print(str_pat.findall(text2))

str_pat = re.compile(r'"(.*?)"')
print(str_pat.findall(text2))

['no.']
['no." Phone says "yes.']
['no.', 'yes.']


#### 4.多行匹配模式

In [6]:
comment = re.compile(r'/\*(.*?)\*/')
text1 = '/* this is a comment */'
text2 = '''/* this is a
... multiline comment */
... '''

print(comment.findall(text1))
print(comment.findall(text2))

comment = re.compile(r'/\*((?:.|\n)*?)\*/')
#在这个模式中， (?:.|\n) 指定了一个非捕获组 (也就是它定义了一个仅仅用来做匹配，而不能通过单独捕获或者编号的组)。
print(comment.findall(text2))

comment = re.compile(r'/\*(.*?)\*/', re.DOTALL)
#re.DOTALL 可以使得.匹配换行符
print(comment.findall(text2))

[' this is a comment ']
[]
[' this is a\n... multiline comment ']
[' this is a\n... multiline comment ']


#### 5.将Unicode文本标准化
在需要比较字符串的程序中使用字符的多种表示会产生问题。 为了修正这个问题，你可以使用unicodedata模块先将文本标准化：

In [7]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'
print(s1)
print(s2)
print(s1 == s2)
print(len(s1))
print(len(s2))

Spicy Jalapeño
Spicy Jalapeño
False
14
15


In [8]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'
import unicodedata
t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)
print(t1 == t2)
print(ascii(t1))

t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)
print(t3 == t4)
print(ascii(t3))

True
'Spicy Jalape\xf1o'
True
'Spicy Jalapen\u0303o'


normalize() 第一个参数指定字符串标准化的方式。 NFC表示字符应该是整体组成(比如可能的话就使用单一编码)，而NFD表示字符应该分解为多个组合字符表示。
去除变音字符：

In [9]:
s1 = 'Spicy Jalape\u00f1o'
t1 = unicodedata.normalize('NFD', s1)
''.join(c for c in t1 if not unicodedata.combining(c))

'Spicy Jalapeno'

#### 6.审查清理文本字符串

In [12]:
s = 'pýtĥöñ\fis\tawesome\r\n'
remap = {
    ord('\t') : ' ',
    ord('\f') : ' ',
    ord('\r') : None # Deleted
}
a = s.translate(remap)

import unicodedata
import sys
cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode) if unicodedata.combining(chr(c)))

b = unicodedata.normalize('NFD', a)
print(b)
print(b.translate(cmb_chrs))

pýtĥöñ is awesome

python is awesome



#### 7.字符串连接
尽量少使用 + 操作，因为每一次执行+=操作的时候会创建一个新的字符串对象，会引起内存复制以及垃圾回收操作。
**最好使用生成器**

In [13]:
data = ['ACME', 50, 91.1]
','.join(str(d) for d in data)

'ACME,50,91.1'

In [ ]:
print(a + ':' + b + ':' + c) # Ugly
print(':'.join([a, b, c])) # Still ugly
print(a, b, c, sep=':') # Better

#### 8. 字符串中插入变量

In [14]:
s = '{name} has {n} messages.'
s.format(name='Guido', n=37)

'Guido has 37 messages.'

In [17]:
class safesub(dict):
    """防止key找不到"""
    def __missing__(self, key):
        return '{' + key + '}'

s = '{name} has {n} messages.'
import sys

def sub(text):
    return text.format_map(safesub(sys._getframe(1).f_locals))

name = 'Guido'
n = 37
print(sub('Hello {name}'))
print(sub('You have {n} messages.'))
print(sub('Your favorite color is {color}'))

Hello Guido
You have 37 messages.
Your favorite color is {color}


#### 9.以指定列宽格式化字符串

In [25]:
s = "Look into my eyes, look into my eyes, the eyes, the eyes, \
the eyes, not around the eyes, don't look around the eyes, \
look into my eyes, you're under."

import textwrap
print(textwrap.fill(s, 70))
print(textwrap.fill(s, 40))
print(textwrap.fill(s, 40, initial_indent='    '))
print(textwrap.fill(s, 40, subsequent_indent='    '))

import os
#使用 os.get_terminal_size() 方法来获取终端的大小尺寸
print(textwrap.fill(s, os.get_terminal_size().columns, initial_indent='    '))

Look into my eyes, look into my eyes, the eyes, the eyes, the eyes,
not around the eyes, don't look around the eyes, look into my eyes,
you're under.
Look into my eyes, look into my eyes,
the eyes, the eyes, the eyes, not around
the eyes, don't look around the eyes,
look into my eyes, you're under.
    Look into my eyes, look into my
eyes, the eyes, the eyes, the eyes, not
around the eyes, don't look around the
eyes, look into my eyes, you're under.
Look into my eyes, look into my eyes,
    the eyes, the eyes, the eyes, not
    around the eyes, don't look around
    the eyes, look into my eyes, you're
    under.
    Look into my eyes, look into my eyes, the eyes, the eyes, the eyes, not
around the eyes, don't look around the eyes, look into my eyes, you're under.
